<a href="https://colab.research.google.com/github/cinqbox/liquipedia-esports/blob/main/liquipedia%E3%80%80%E5%8F%8E%E9%9B%86%E7%94%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
import requests
import json
import pprint
import time
import re
import pandas as pd

# 結果まとめ関数
def result(pattern, string):
  ans = "".join(re.findall(pattern, tes, flags=re.MULTILINE))
  return ans



# ゲームタイトル
game_title = "valorant"
S = requests.Session()
url = "https://liquipedia.net/" + game_title + "/api.php"

# 日本大会の取得
params = {"action": "query",
          "format": "json",
          "list": "categorymembers",
          "cmprop": "title",
          "cmlimit": "100",
          "cmtitle": "Category:Japanese Tournaments"}

R = S.get(url=url, params=params)
DATA = R.json()
print(R)
pprint.pprint(DATA)
time.sleep(10)

<Response [200]>
{'batchcomplete': '',
 'query': {'categorymembers': [{'ns': 0,
                                'title': 'A.W EXTREME MASTERS/Pro '
                                         'Invitational'},
                               {'ns': 0, 'title': 'BOSS ARENA'},
                               {'ns': 2, 'title': 'User:CarterPH/evc'},
                               {'ns': 0, 'title': 'E-Region Cup/0'},
                               {'ns': 0, 'title': 'E-Region Cup/1'},
                               {'ns': 0, 'title': 'EDION Valorant Cup'},
                               {'ns': 0,
                                'title': 'Fiveness Japan Cup VALORANT 2020'},
                               {'ns': 0,
                                'title': 'GALLERIA GLOBAL CHALLENGE/2020'},
                               {'ns': 0, 'title': 'Immortality Cup'},
                               {'ns': 0,
                                'title': 'Joshin CUP/2020/SUMMER TOURNAMENT'},
                    

In [33]:
# タイトル一覧を抽出
title_list = []
title = DATA["query"]["categorymembers"]
for i in title:
  title_list.append(i["title"])
  print(i["title"])

A.W EXTREME MASTERS/Pro Invitational
BOSS ARENA
User:CarterPH/evc
E-Region Cup/0
E-Region Cup/1
EDION Valorant Cup
Fiveness Japan Cup VALORANT 2020
GALLERIA GLOBAL CHALLENGE/2020
Immortality Cup
Joshin CUP/2020/SUMMER TOURNAMENT
Joshin CUP/2020/SUMMER TOURNAMENT/Qualifier/1
Mildom Masters
RAGE Invitational
RAGE Japan Tournament
RAGE Japan Tournament/Open Qualifier
Rock de Nashi CUP/1
Twitch Rivals/Launch Showdown/Japan
UTAGE VALORANT/Season 1
VALORANT from TOKYO/1
VALORANT OPEN ASIA


In [34]:
# 結果まとめシート
result_df = pd.DataFrame()

for title in title_list:
  # データの取得
  params = {"action": "query",
          "format": "json",
          "prop": "revisions",
          "rvprop": "content|ids",
          "titles": title}

  R = S.get(url=url, params=params)
  DATA = R.json()
  ids = list(DATA["query"]["pages"].keys())[0]
  # print(ids)
  # print(R)
  # pprint.pprint(DATA)

  tes = DATA["query"]["pages"][ids]["revisions"][0]['*']

  pattern_prize = r"^\|prizepool=(.*?)$"
  pattern_type = r"^\|type=(.*?)$"
  pattern_country = r"^\|country=(.*?)$"
  pattern_prize_local = r"^\|localcurrency=(.*?)$"
  pattern_team_number = r"^\|team_number=(.*?)$"
  pattern_name = r"^\|name=(.*?)$"
  pattern_prize_usd = r"^\|prizepoolusd=(.*?)$"


  prize_local = result(pattern_prize_local, tes)
  holdtype = result(pattern_type, tes)
  team_number = result(pattern_team_number, tes)
  country = result(pattern_country, tes)
  name = result(pattern_name, tes)

  if prize_local == "":
    prize = result(pattern_prize_usd, tes)
    prize_local = "usd"
  else:
    prize = result(pattern_prize, tes)


  dic = {"大会名": name, "賞金": prize, "通貨": prize_local, "開催方法": holdtype, "チーム数": team_number, "開催地域": country}

  # print(dic)

  # データの整形、結合
  df = pd.DataFrame.from_dict(dic, orient='index').T
  print(df)
  result_df = pd.concat([result_df, df], ignore_index=True)
  
  # 加重アクセス防止
  time.sleep(15)


                                    大会名         賞金   通貨    開催方法 チーム数   開催地域
0  A.W EXTREME MASTERS Pro Invitational  1,000,000  JPY  Online   16  Japan
          大会名 賞金   通貨    開催方法 チーム数   開催地域
0  BOSS ARENA     usd  Online    2  Japan
                  大会名         賞金   通貨     開催方法 チーム数   開催地域
0  EDION Valorant Cup  1,000,000  jpy  Offline    6  Japan
               大会名      賞金   通貨    開催方法 チーム数   開催地域
0  e-Region Cup #0  30,000  jpy  Online    4  Japan
               大会名      賞金   通貨    開催方法 チーム数   開催地域
0  e-Region Cup #1  30,000  jpy  Online    4  Japan
                  大会名         賞金   通貨     開催方法 チーム数   開催地域
0  EDION Valorant Cup  1,000,000  jpy  Offline    6  Japan
                                大会名      賞金   通貨    開催方法 チーム数   開催地域
0  Fiveness Japan Cup VALORANT 2020  50,000  jpy  Online   28  Japan
                              大会名         賞金   通貨    開催方法 チーム数   開催地域
0  GALLERIA GLOBAL CHALLENGE 2020  1,000,000  JPY  Online   16  Japan
               大会名      賞金   通貨    開催方法 チー

In [35]:

display(result_df)

,大会名,賞金,通貨,開催方法,チーム数,開催地域
0,A.W EXTREME MASTERS Pro Invitational,"1,000,000",JPY,Online,16,Japan
1,BOSS ARENA,,usd,Online,2,Japan
2,EDION Valorant Cup,"1,000,000",jpy,Offline,6,Japan
3,e-Region Cup #0,"30,000",jpy,Online,4,Japan
4,e-Region Cup #1,"30,000",jpy,Online,4,Japan
5,EDION Valorant Cup,"1,000,000",jpy,Offline,6,Japan
6,Fiveness Japan Cup VALORANT 2020,"50,000",jpy,Online,28,Japan
7,GALLERIA GLOBAL CHALLENGE 2020,"1,000,000",JPY,Online,16,Japan
8,Immortality Cup,"50,000",jpy,Online,4,Japan
9,Joshin CUP VALORANT 2020 SUMMER TOURNAMENT,"100,000",JPY,Online,2,Japan


In [36]:
import numpy as np
result_df.replace("", np.nan, inplace=True)

In [37]:
print(len(result_df))

20


In [38]:
result_df["通貨"] = result_df["通貨"].str.upper()
result_df = result_df.dropna(how="any")
result_df = result_df[~result_df["大会名"].duplicated()]
print(len(result_df))
result_df

16


,大会名,賞金,通貨,開催方法,チーム数,開催地域
0,A.W EXTREME MASTERS Pro Invitational,"1,000,000",JPY,Online,16,Japan
2,EDION Valorant Cup,"1,000,000",JPY,Offline,6,Japan
3,e-Region Cup #0,"30,000",JPY,Online,4,Japan
4,e-Region Cup #1,"30,000",JPY,Online,4,Japan
6,Fiveness Japan Cup VALORANT 2020,"50,000",JPY,Online,28,Japan
7,GALLERIA GLOBAL CHALLENGE 2020,"1,000,000",JPY,Online,16,Japan
8,Immortality Cup,"50,000",JPY,Online,4,Japan
9,Joshin CUP VALORANT 2020 SUMMER TOURNAMENT,"100,000",JPY,Online,2,Japan
11,Mildom Masters,"2,500,000",JPY,Online,8,Japan
12,RAGE Invitational,"500,000",JPY,Online,16,Japan


In [39]:
result_df.to_csv(game_title + ".csv", encoding="utf-8-sig", index=False)